<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Reading-data" data-toc-modified-id="Reading-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Reading data</a></span></li><li><span><a href="#Prepartion-of-the-spectra" data-toc-modified-id="Prepartion-of-the-spectra-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepartion of the spectra</a></span></li><li><span><a href="#Host-galaxy-subtraction" data-toc-modified-id="Host-galaxy-subtraction-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Host-galaxy subtraction</a></span></li><li><span><a href="#Creation-of-the-predefined-lists-of-emission-lines" data-toc-modified-id="Creation-of-the-predefined-lists-of-emission-lines-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Creation of the predefined lists of emission lines</a></span></li><li><span><a href="#Defining-the-fitting-model" data-toc-modified-id="Defining-the-fitting-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Defining the fitting model</a></span></li><li><span><a href="#Fittings" data-toc-modified-id="Fittings-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Fittings</a></span></li><li><span><a href="#Plotting-the-results" data-toc-modified-id="Plotting-the-results-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Plotting the results</a></span></li><li><span><a href="#Inspecting-&amp;-saving-the-plotting-results" data-toc-modified-id="Inspecting-&amp;-saving-the-plotting-results-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Inspecting &amp; saving the plotting results</a></span></li><li><span><a href="#Simple-analysis" data-toc-modified-id="Simple-analysis-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Simple analysis</a></span></li></ul></div>

**Nicolás Guerra-Varas**

Prof. Dragana Ilić

Tutor Isidora Jankov

# FANTASY Tutorial

The goal of this tutorial is to learn how to use __[`fantasy`](https://fantasy-agn.readthedocs.io/en/latest/index.html)__ to model AGN spectra.

## Imports

In [1]:
%matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

import numpy as np
import pandas as pd

import glob

Using matplotlib backend: MacOSX


In [2]:
# additional imports
import math as m
import csv
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import ascii
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from astropy.table import Table

In [3]:
from fantasy_agn.tools import read_sdss, read_text, read_gama_fits

from fantasy_agn.models import (create_input_folder, create_feii_model, create_model, 
                               create_tied_model, continuum, create_line, create_fixed_model)

# Replicate Procedure

## Read spectrum

In [4]:
# assigned spectrum
s = read_sdss('spec-0572-52289-0442.fits')
s_or = read_sdss('spec-0572-52289-0442.fits')
# saving original spectrum to see how it changes through the procedure

# these are the parameters that the spectrum has
# s.z redshift
# s.ra
# s.dec
# s.mjd
# s.plate
# s.fiber
# s.name
# s.err
# s.flux
# s.wave
# s.fwhm
# s.velscale

First I took a quick look into the spectrum.

In [5]:
plt.style.context(['nature', 'notebook'])
plt.figure(figsize=(12,5))

plt.plot(s.wave, s.flux, color='navy', label='Obs', lw=1)

plt.legend(loc='upper right', prop={'size': 15}, frameon=False, ncol=2)
plt.xlabel(r'Wavelength [Ang]')
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]')

# plt.savefig('spectrum_original.pdf')

plt.show()

I looked it up on SDSS/SIMBAD to get more information on this object.

In [6]:
print(s.plate, s.mjd, s.fiber)

572 52289 442


Here's an image of the object from the __[SDSS viewer](https://skyserver.sdss.org/dr18/VisualTools/explore/summary?plate=0572&mjd=52289&fiber=442)__

<img src='sdss_viewer.jpg' width="400" height="400">

Then I wanted to see where in the sky this is.

In [7]:
ra = s.ra
dec = s.dec
eq = SkyCoord(ra, dec, frame='galactic', unit=u.deg)
gal = eq.galactic

plt.style.use('ggplot')

fig = plt.figure(figsize=(6,5))
plt.subplot(111, projection='aitoff')

plt.plot(gal.l.wrap_at(180*u.deg), gal.b.wrap_at(180*u.deg), linestyle='None')
plt.scatter(gal.l.wrap_at('180d').radian, gal.b.radian, marker='*', color='navy', s=150)

plt.grid(True, alpha=0.5)

plt.tick_params(axis='both', which='major', labelsize=5)

# plt.savefig('map_qso.pdf')

plt.show()

It is along the galactic plane. Maybe it will have significant Galactic absorption.

In [8]:
print('Redshift =', s.z)

Redshift = 0.16176972


Finally, I checked its redshift, which indicates it is a distant object.

## Prepare spectra

In [9]:
s.DeRedden()  # correct for Galactic extinction
s.CorRed()  # correct for cosmological redshift

I corrected for redshift and extinction and plotted against the unchanged data to see how it changes.

In [10]:
plt.style.context(['nature', 'notebook'])
plt.figure(figsize=(12,5))

plt.plot(s_or.wave, s_or.flux, '--', color='steelblue', label='Obs Original', lw=1)
plt.plot(s.wave, s.flux, color='navy', label='Obs Corrected', lw=1)

plt.legend(loc='upper right', prop={'size': 15}, frameon=False, ncol=1)
plt.xlabel(r'Wavelength [Ang]')
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]')

# plt.savefig('spectrum_pre_post_corrections.pdf')

plt.show()

## Host-galaxy subtraction

In [11]:
# s.restore()
s.fit_host_sdss()  # fits for the host galaxy and AGN contibutions separately

s.host_no_mask = s.host  # saving the host

plt.legend(fontsize=12)
plt.xlabel(r'Wavelength [Ang]')
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]')
plt.title('Unmasked Host')

# plt.savefig('fit_host_sdss_default.pdf')

plt.show()

Here, the spectrum in red corresponds to the data. The blue and purple spectra are the fits to the host galaxy and the AGN respectively. Their summed spectrum gives the fit to the observations, which is quite good. The contribution of the host galaxy in this case is not negligible.

In [12]:
s.restore()
s.fit_host_sdss(mask_host=True, custom=False)

s.host_masked = s.host  # saving the host

plt.legend(fontsize=12)
plt.xlabel(r'Wavelength [Ang]')
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]')
plt.title('Masked Host')

# plt.savefig('fit_host_sdss_masked.pdf')

plt.show()

I tried the host subtraction again, but now masking the main emission lines from the host galaxy ([O III], [N II], [S II]). The code does this by interpolating between the two edges of the lines instead of going through the line itself. The result was very different (see above). Now, the contribution of the host seems much less significant. The fit overall seems very good, just like the first try.

In [13]:
plt.style.context(['nature', 'notebook'])
plt.figure(figsize=(12,5))

plt.plot(s.wave, s.flux, color='navy', label='Obs Corrected', lw=1)

plt.legend(loc='upper left', prop={'size': 15}, frameon=False, ncol=1)
plt.xlabel(r'Wavelength [Ang]')
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]')

# plt.savefig('spectrum_post_host_sub.pdf')

plt.show()

In [14]:
plt.style.context(['nature', 'notebook'])
plt.figure(figsize=(12, 5))

plt.plot(s.wave, s.flux+s.host, color='navy', label='Obs' , lw=1.5)
plt.plot(s.wave, s.host_no_mask, color="#F10C45", label='Host: not masked', lw=1.0)
plt.plot(s.wave, s.host, color="green", label='Host: masked', lw=1.0)

plt.legend(loc='upper left', prop={'size': 10}, frameon=False)
plt.xlabel(r'Wavelength [Ang]')
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]')

# plt.savefig('spectrum_host_masked_vs_not.pdf')

plt.show()

In this plot, it is clear that the difference between both host fits are the lines.

## Creation of the predefined lists of emission lines

In [15]:
# this is the minimum wavelength found in the spectrum
np.min(s.wave)

3450.3032

In [16]:
# the information of the lines will be saved in the 'lines' folder
create_input_folder(xmin=np.min(s.wave), xmax=np.max(s.wave), path_to_folder='lines/', overwrite=True)

Directory  lines/  already exists


In this folder, there's a file for each of these lines, commonly found in AGN spectra: Balmer series [FeII], He, narrow lines, broad lines, [OIII], and others such as a broad and narrow line model.

In [17]:
# cropping to a certain wavelength range

# I won't crop it initially
# s.crop(np.min(s.wave), np.max(s.wave))

# I'm cropping it for the 2nd attempt on
s.crop(3575, np.max(s.wave))

### Wavelengths of the lines

In [18]:
H_lines = pd.read_csv('lines/hydrogen.csv')
O_N_lines = pd.read_csv('lines/oiii_nii.csv')
narrow_lines = pd.read_csv('lines/narrow_basic.csv')
narrowP = pd.read_csv('lines/narrow_plus.csv')

In [19]:
H_alpha = np.float64(H_lines.loc[H_lines['line']=='Ha']['position'])
H_beta = np.float64(H_lines.loc[H_lines['line']=='Hb']['position'])

OIII_1 = np.float64(O_N_lines.loc[O_N_lines['line']=='[OIII]']['position'][0])
OIII_2 = np.float64(O_N_lines.loc[O_N_lines['line']=='[OIII]']['position'][1])
OIII_narrow = np.float64(narrow_lines.loc[narrow_lines['line']=='[OIII]']['position'])

NII_1 = np.float64(O_N_lines.loc[O_N_lines['line']=='[NII]']['position'][2])
NII_2 = np.float64(O_N_lines.loc[O_N_lines['line']=='[NII]']['position'][3])

SII_narrow_1 = np.float64(narrow_lines.loc[narrow_lines['line']=='[SII]']['position'][6])
SII_narrow_2 = np.float64(narrow_lines.loc[narrow_lines['line']=='[SII]']['position'][7])
SII_narrowP_1 = np.float64(narrowP.loc[narrowP['line']=='[SII]1F']['position'][7])
SII_narrowP_2 = np.float64(narrowP.loc[narrowP['line']=='[SII]1F']['position'][8])

## Defining the fitting model

In [20]:
# continumm
cont_1 = continuum(s, min_refer=5690, refer=5700, max_refer=5710)

cont_2 = continuum(s, 
                   # min_refer=4040, refer=4050, max_refer=4060, 
                   min_refer=4190, refer=4200, max_refer=4210, # best
                   # index1=-1.0, 
                   # index2=-1.5, min_index2=-2.5, max_index2=-0.5)
                   # index2=-2, min_index2=-3, max_index2=-1)
                   index2=-2.5, min_index2=-3.5, max_index2=-1.5)  # best
                   # index2=-3, min_index2=-4, max_index2=-2)

# default parameters:
# refer=5500, min_refer=5400, max_refer=7000
# index1=-1.7, min_index1=-3, max_index1=0
# index2=0, min_index2=-1, max_index2=1

# index1: sets the power law of the continuum
# index2: makes the continuum flat => I modified it

In [21]:
# broad line component
broad_1 = create_fixed_model(['hydrogen.csv'], name='Broad')

In [22]:
# narrow line component
narrow_1 = create_tied_model(name='OIII5007', 
                           files=['narrow_basic.csv', 'hydrogen.csv'], 
                           prefix='nr', 
                           fwhm=1000, 
                           min_offset=0, 
                           max_offset=300, 
                           min_fwhm=900, 
                           max_fwhm=1200, 
                           fix_oiii_ratio=True, 
                           position=5006.803341, 
                           included=True, 
                           min_amplitude=0.2)

In [23]:
# helium
he_1 = create_fixed_model(['helium.csv'], name='He', fwhm=3000, min_fwhm=1000, max_fwhm=5000)

# He.amp_HeI_4144 is at its minimum boundary 0.0
# He.amp_HeI_4471 is at its minimum boundary 0.0
# He.fwhm is at its maximum boundary 5000.0

In [24]:
# iron
fe_1 = create_feii_model(name='feii', 
                         fwhm=1800, 
                         min_fwhm=1000, 
                         max_fwhm=2000, 
                         offset=0,
                         min_offset=-3000, 
                         max_offset=3000)

fe_2 = create_feii_model(name='feii', 
                         fwhm=4000,  # best
                         # fwhm=1800, 
                         # fwhm=2200, 
                         min_fwhm=1000, 
                         max_fwhm=8000, 
                         offset=0,  # best
                         # offset=20,
                         min_offset=-3000, 
                         max_offset=3000)

In [25]:
# Code fits simultaneously all features.
model_1st = cont_1 + broad_1 + narrow_1 + fe_1 + he_1
model_2nd = cont_2 + broad_1 + narrow_1 + fe_1 + he_1
model_3rd = cont_2 + broad_1 + narrow_1 + fe_2 + he_1

So, I am trying different attempts:
1. No modifications
2. Fixing the continuum
3. Fixing the iron lines

## Fitting

In [26]:
# 1st attempt
# s.fit(model_1st, ntrial=2)

# 2nd attempt: fixing continuum
# s.fit(model_2nd, ntrial=3)

# 3rd attempt: fixing lines
s.fit(model_3rd, ntrial=3)

stati 217854.71219723186
1 iter stat:  11.320828548572536
2 iter stat:  3.8425007299410936
3 iter stat:  1.680778738716981


In [27]:
# it converged
print(s.gres.succeeded)

True


In [28]:
# saving info on each fit attempt
attempts = {'chi2 1st': 124.23362044251917, 'iters 1st': 2, 
            'chi2 2nd': 3.6474814654284273, 'iters 2nd': 3, 
            'chi2 3rd': 1.680778738716981, 'iters 3rd': 3,
            }

##  Inspecting & saving the plotting results

__[`sherpa`](https://sherpa.readthedocs.io/en/4.15.0/)__ saves the model's output into `gres`

In [29]:
arr = np.stack([np.asarray(s.gres.parnames), np.asarray(s.gres.parvals)], axis=-1)
columns = ['parnames', 'parvalues']

fit_result = pd.DataFrame(arr, columns=columns)
fit_result

,parnames,parvalues
0,brokenpowerlaw.refer,4190.0
1,brokenpowerlaw.ampl,66.18675958054816
2,brokenpowerlaw.index1,-2.057828498535581
3,brokenpowerlaw.index2,-3.3775181230255793
4,Broad.amp_Heps_3970,0.0
5,Broad.amp_Hd_4102,12.732502498874283
6,Broad.amp_Hg_4340,31.07817992852238
7,Broad.amp_Hb_4861,61.387900230920486
8,Broad.amp_Ha_6563,104.59213829824655
9,Broad.offs_kms,178.9540263118911


In [30]:
# model_1st
# model_2nd
model_3rd

<BinaryOpModel model instance '((((brokenpowerlaw + Broad) + ((((((((((((((((nr_OIII5007 + nr_OIII4958) + nr_NII6584) + nr_NIII6548) + nr_[OII]_3726) + nr_[OII]_3728) + nr_[NeIII]_3868) + nr_[OIII]_4363) + nr_[OI]_6300) + nr_[OI]_6363) + nr_[SII]_6716) + nr_[SII]_6730) + nr_Heps_3970) + nr_Hd_4101) + nr_Hg_4340) + nr_Hb_4861) + nr_Ha_6562)) + feii) + He)'>

In [31]:
# To see the fitting results, one can use standard outputs from Sherpa package, such as:
# gres - to list all fitting results
# gres.parnames - to list all parameters
# save_json() - to save the results

print(s.gres.format())
s.save_json() #saving parameters 

Method                = levmar
Statistic             = chi2
Initial fit statistic = 8296.83
Final fit statistic   = 8295.15 at function evaluation 752
Data points           = 3033
Degrees of freedom    = 2982
Probability [Q-value] = 0
Reduced statistic     = 2.78174
Change in statistic   = 1.68078
   brokenpowerlaw.refer   4190         +/- 7.44266     
   brokenpowerlaw.ampl   66.1868      +/- 0.546797    
   brokenpowerlaw.index1   -2.05783     +/- 0.0145207   
   brokenpowerlaw.index2   -3.37752     +/- 0.0433846   
   Broad.amp_Heps_3970   0            +/- 0.590603    
   Broad.amp_Hd_4102   12.7325      +/- 0.671117    
   Broad.amp_Hg_4340   31.0782      +/- 0.830577    
   Broad.amp_Hb_4861   61.3879      +/- 0.577522    
   Broad.amp_Ha_6563   104.592      +/- 1.00091     
   Broad.offs_kms   178.954      +/- 7.47722     
   Broad.fwhm     4880.1       +/- 24.1331     
   nr_OIII5007.ampl   41.692       +/- 0.726013    
   nr_OIII5007.offs_kms   15.352       +/- 5.33003     
   

In the first try, I didn't modify the parameters of the model. I got some warnings:
```
WARNING: parameter value feii.fwhm is at its maximum boundary 2000.0
WARNING: parameter value He.amp_HeI_4144 is at its minimum boundary 0.0
WARNING: parameter value He.amp_HeI_4471 is at its minimum boundary 0.0
WARNING: parameter value He.fwhm is at its maximum boundary 5000.0
```

I tried tuned the parameters of the model until they were resolved. Warning of the same type also came up during the second attempt.

---

I plotted the spectra with spectral lines indicated to get a better understanding of how well the model is fitting the data.

## Plotting the results

In [32]:
'''plt.style.context(['nature', 'notebook'])
plt.figure(figsize=(18,8))

plt.plot(s.wave, s.flux, color="#929591", label='Obs', lw=2)
plt.plot(s.wave, model_1st(s.wave), color="#F10C45", label='Model', lw=3)
plt.plot(s.wave, model_1st(s.wave)-s.flux-70, '-', color="#929591", label='Residual', lw=2)
plt.axhline(y=-70, color='deepskyblue', linestyle='--', lw=2)

plt.plot(s.wave, cont_1(s.wave),'--', color="#042E60", label='Continuum', lw=1.5)
plt.plot(s.wave, narrow_1(s.wave), label='Narrow', color="#25A36F", lw=3)
plt.plot(s.wave, broad_1(s.wave), label='Broad H', lw=3, color="#2E5A88")
plt.plot(s.wave, he_1(s.wave), label='Broad He I', lw=3, color="orange")
plt.plot(s.wave, fe_1(s.wave), '-', color="#CB416B", label='Fe II model', lw=3)

plt.xlabel(r'Wavelength [Ang]', fontsize=15)
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]', fontsize=15)

plt.xlim(np.min(s.wave), np.max(s.wave))
plt.tick_params(which='both', direction="in")
plt.yticks(fontsize=15)
plt.xticks(np.arange(4000, 7000, step=500), fontsize=15)
plt.title('Fantasy Fit: 1st Attempt (Iters: ' + str(attempts['iters 1st']) + r', $ \chi ^2 $ = ' + str(np.round(attempts['chi2 1st'], 3)) + ')')

plt.legend(loc='upper left',  prop={'size': 12}, frameon=False, ncol=2)

# plt.savefig('1st_attempt_fantasy_fit.pdf')

plt.show()''';

In [33]:
'''plt.style.context(['nature', 'notebook'])
plt.figure(figsize=(18,8))

plt.plot(s.wave, s.flux, color="#929591", label='Obs', lw=2)
plt.plot(s.wave, model_2nd(s.wave), color="#F10C45", label='Model', lw=3)
plt.plot(s.wave, model_2nd(s.wave)-s.flux-70, '-', color="#929591", label='Residual', lw=2)
plt.axhline(y=-70, color='deepskyblue', linestyle='--', lw=2)

plt.plot(s.wave, cont_2(s.wave),'--', color="#042E60", label='Continuum', lw=1.5)
plt.plot(s.wave, narrow_1(s.wave), label='Narrow', color="#25A36F", lw=3)
plt.plot(s.wave, broad_1(s.wave), label='Broad H', lw=3, color="#2E5A88")
plt.plot(s.wave, he_1(s.wave), label='Broad He I', lw=3, color="orange")
plt.plot(s.wave, fe_1(s.wave), '-', color="#CB416B", label='Fe II model', lw=3)

plt.xlabel(r'Wavelength [Ang]', fontsize=15)
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]', fontsize=15)

plt.xlim(np.min(s.wave), np.max(s.wave))
plt.tick_params(which='both', direction="in")
plt.yticks(fontsize=15)
plt.xticks(np.arange(4000, 7000, step=500), fontsize=15)
plt.title('Fantasy Fit: 2nd Attempt (Iters: ' + str(attempts['iters 2nd']) + r', $ \chi ^2 $ = ' + str(np.round(attempts['chi2 2nd'], 3)) + ')')

plt.legend(loc='upper left',  prop={'size': 12}, frameon=False, ncol=2)

# plt.savefig('2nd_attempt_fantasy_fit.pdf')

plt.show()'''

'plt.style.context([\'nature\', \'notebook\'])\nplt.figure(figsize=(18,8))\n\nplt.plot(s.wave, s.flux, color="#929591", label=\'Obs\', lw=2)\nplt.plot(s.wave, model_2nd(s.wave), color="#F10C45", label=\'Model\', lw=3)\nplt.plot(s.wave, model_2nd(s.wave)-s.flux-70, \'-\', color="#929591", label=\'Residual\', lw=2)\nplt.axhline(y=-70, color=\'deepskyblue\', linestyle=\'--\', lw=2)\n\nplt.plot(s.wave, cont_2(s.wave),\'--\', color="#042E60", label=\'Continuum\', lw=1.5)\nplt.plot(s.wave, narrow_1(s.wave), label=\'Narrow\', color="#25A36F", lw=3)\nplt.plot(s.wave, broad_1(s.wave), label=\'Broad H\', lw=3, color="#2E5A88")\nplt.plot(s.wave, he_1(s.wave), label=\'Broad He I\', lw=3, color="orange")\nplt.plot(s.wave, fe_1(s.wave), \'-\', color="#CB416B", label=\'Fe II model\', lw=3)\n\nplt.xlabel(r\'Wavelength [Ang]\', fontsize=15)\nplt.ylabel(r\'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]\', fontsize=15)\n\nplt.xlim(np.min(s.wave), np.max(s.wave))\nplt.tick_params(which=\'both\', direction="in")\

In [34]:
'''plt.style.context(['nature', 'notebook'])
plt.figure(figsize=(18,8))

plt.plot(s.wave, s.flux, color="#929591", label='Obs', lw=2)
plt.plot(s.wave, model_3rd(s.wave), color="#F10C45", label='Model', lw=3)
plt.plot(s.wave, model_3rd(s.wave)-s.flux-70, '-', color="#929591", label='Residual', lw=2)
plt.axhline(y=-70, color='deepskyblue', linestyle='--', lw=2)

plt.plot(s.wave, cont_2(s.wave),'--', color="#042E60", label='Continuum', lw=1.5)
plt.plot(s.wave, narrow_1(s.wave), label='Narrow', color="#25A36F", lw=3)
plt.plot(s.wave, broad_1(s.wave), label='Broad H', lw=3, color="#2E5A88")
plt.plot(s.wave, he_1(s.wave), label='Broad He I', lw=3, color="orange")
plt.plot(s.wave, fe_2(s.wave), '-', color="#CB416B", label='Fe II model', lw=3)

plt.xlabel(r'Wavelength [Ang]', fontsize=15)
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]', fontsize=15)

plt.xlim(np.min(s.wave), np.max(s.wave))
plt.tick_params(which='both', direction="in")
plt.yticks(fontsize=15)
plt.xticks(np.arange(4000, 7000, step=500), fontsize=15)
plt.title('Fantasy Fit: 3rd Attempt (Iters: ' + str(attempts['iters 3rd']) + r', $ \chi ^2 $ = ' + str(np.round(attempts['chi2 3rd'], 3)) + ')')

plt.legend(loc='upper left',  prop={'size': 12}, frameon=False, ncol=2)

# plt.savefig('3rd_attempt_fantasy_fit.pdf')

plt.show()'''

'plt.style.context([\'nature\', \'notebook\'])\nplt.figure(figsize=(18,8))\n\nplt.plot(s.wave, s.flux, color="#929591", label=\'Obs\', lw=2)\nplt.plot(s.wave, model_3rd(s.wave), color="#F10C45", label=\'Model\', lw=3)\nplt.plot(s.wave, model_3rd(s.wave)-s.flux-70, \'-\', color="#929591", label=\'Residual\', lw=2)\nplt.axhline(y=-70, color=\'deepskyblue\', linestyle=\'--\', lw=2)\n\nplt.plot(s.wave, cont_2(s.wave),\'--\', color="#042E60", label=\'Continuum\', lw=1.5)\nplt.plot(s.wave, narrow_1(s.wave), label=\'Narrow\', color="#25A36F", lw=3)\nplt.plot(s.wave, broad_1(s.wave), label=\'Broad H\', lw=3, color="#2E5A88")\nplt.plot(s.wave, he_1(s.wave), label=\'Broad He I\', lw=3, color="orange")\nplt.plot(s.wave, fe_2(s.wave), \'-\', color="#CB416B", label=\'Fe II model\', lw=3)\n\nplt.xlabel(r\'Wavelength [Ang]\', fontsize=15)\nplt.ylabel(r\'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]\', fontsize=15)\n\nplt.xlim(np.min(s.wave), np.max(s.wave))\nplt.tick_params(which=\'both\', direction="in")\

Next, I created an interactive plot marking the spectral lines so that I can zoom in and see how they are fitting in more detail.

In [35]:
wavelengths = dict({SII_narrowP_1:'[SII] Narrow+, 1', 
                    SII_narrowP_2:'[SII] Narrow+, 2', 
                    OIII_narrow:'[OIII] Narrow', 
                    OIII_1:'[OIII], 1', 
                    OIII_2:'[OIII], 2', 
                    SII_narrow_1:'[SII] Narrow, 1', 
                    SII_narrow_2:'[SII] Narrow, 2',
                    NII_1:'[NII], 1', 
                    NII_2:'[NII], 2', 
                    H_alpha:'H alpha', 
                    H_beta:'H beta'})

colors = ['darkgreen', 'crimson', 'teal', 'goldenrod', 'slateblue', 'darkviolet', 'darkorange', 'royalblue', 
          'magenta', 'darkolivegreen', 'brown']

In [38]:
plt.style.context(['nature', 'notebook'])
fig = plt.figure(figsize=(15, 8))
ax = fig.add_subplot()
fig.subplots_adjust(top=0.85)

for i in range(len(wavelengths)):
    ax.text(list(wavelengths.keys())[i], -200, list(wavelengths.values())[i], fontsize=10, rotation='vertical')
    ax.vlines(list(wavelengths.keys())[i], -100, 350, linestyles='dashed', label=list(wavelengths.values())[i], colors=colors[i])

plt.ion()

plt.plot(s.wave, s.flux, color="#929591", label='Obs', lw=2)
plt.plot(s.wave, model_3rd(s.wave), color="#F10C45", label='Model', lw=3)
plt.plot(s.wave, model_3rd(s.wave)-s.flux-70, '-', color="#929591", label='Residual', lw=2)
plt.axhline(y=-70, color='deepskyblue', linestyle='--', lw=2)

plt.plot(s.wave, cont_2(s.wave),'--', color="#042E60", label='Continuum', lw=1.5)
plt.plot(s.wave, narrow_1(s.wave), label='Narrow', color="#25A36F", lw=3)
plt.plot(s.wave, broad_1(s.wave), label='Broad H', lw=3, color="#2E5A88")
plt.plot(s.wave, he_1(s.wave), label='Broad He I', lw=3, color="orange")
plt.plot(s.wave, fe_2(s.wave), '-', color="#CB416B", label='Fe II model', lw=3)

plt.xlabel(r'Wavelength [Ang]', fontsize=15)
plt.ylabel(r'Flux [$10^{-7}$ erg / s / cm$^2$ / Ang]', fontsize=15)

plt.xlim(np.min(s.wave), np.max(s.wave))
plt.ylim(-220, 350)
plt.tick_params(which='both', direction="in")
plt.yticks(fontsize=15)
plt.xticks(np.arange(4000, 7000, step=500), fontsize=15)

plt.legend(bbox_to_anchor=(1.0, 0.8), prop={'size': 10}, frameon=False, ncol=1)

plt.show()

In [37]:
attempts

{'chi2 1st': 124.23362044251917,
 'iters 1st': 2,
 'chi2 2nd': 3.6474814654284273,
 'iters 2nd': 3,
 'chi2 3rd': 1.680778738716981,
 'iters 3rd': 3}

## Comments on each attempt

After the first attempt, I saw that what need better fitting is:
* Continuum: below 4500 Ang, the continuum isn't steep enough
* [OIII] 1 and 2: the iron model doesn't have intense enough lines

### First Attempt
The first time I ran the model just the way it was to have a starting point. I believe the most important thing to fix is the continuum. The $\chi^2$ was equal to $124.234$.

### Second Attempt
I focused on fixing the continuum. I changed the breakpoint of the power law (`refer`) and I made the left side of it steeper with the (`index2`). I also made the model run three times instead of two. The result is much better, it has a $\chi^2$ of $3.647$. During this attempt I realised there is a typo in the source code of fantasy! When creating the continuum model, the condition to choose the power law was wrong. I went to the source and fixed this. This was the only way I was able to improve the fit of the continuum.

### Third Attempt
Then, in the next attempt, I tried to fix the FeII line model, since the lines that needed fixing the most are the first and second [OIII] spectral lines, at 4958.90 and 5006.80 Ang respectively. I did this by increasing the FWHM and its maximum possible value in the `feii` model. This worked and lowered the $\chi^2$ to $1.681$. However, this effect is not so noticeable in the plot.

---

I note that there is a level of randomness to the model. Sherpas computes a Least Squares algorithms, which doesn't always run in the same way. So the $chi^2$ values are not exactly the same every time the code is run, but they are within the same order of magnitude.

# Measuring Lines

Now, I'll measure the flux of the following lines:

* Narrow, if present:
    * [OIII]
    * [NII]
    * [SII]
    * H $\alpha$
    * H $\beta$

* Broad:
    * H $\alpha$
    * H $\beta$

Since my galaxy has a significant contribution from the host galaxy, I'll also measure the flux from the original observed spectrum.

#### H $\alpha$

In [70]:
# H alpha
print('Model: ')

mask_H_alpha = (s.wave > 6400) & (s.wave < 6700)

# broad
flux_Ha_broad = np.sum(broad_1(s.wave)[mask_H_alpha])
print('Broad H alpha = ' + str(flux_Ha_broad) + ' [1e-7 erg / (s cm^2 Ang)]')

# narrow
flux_Ha_narrow = np.sum(narrow_1(s.wave)[mask_H_alpha])
print('Narrow H alpha = ' + str(flux_Ha_narrow) + ' [1e-7 erg / (s cm^2 Ang)]')

Model: 
Broad H alpha = 7860.6343 [1e-7 erg / (s cm^2 Ang)]
Narrow H alpha = 4405.9497 [1e-7 erg / (s cm^2 Ang)]


In [81]:
# H alpha
print('Observed spectrum: ')

mask_H_alpha_or = (s_or.wave > 6400) & (s_or.wave < 6700)

# broad
flux_Ha_broad_or = np.sum(broad_1(s_or.wave)[mask_H_alpha_or])
print('Broad H alpha = ' + str(flux_Ha_broad_or) + ' [1e-7 erg / (s cm^2 Ang)]')

# narrow
flux_Ha_narrow_or = np.sum(narrow_1(s_or.wave)[mask_H_alpha_or])
print('Narrow H alpha = ' + str(flux_Ha_narrow_or) + ' [1e-7 erg / (s cm^2 Ang)]')

Observed spectrum: 
Broad H alpha = 7859.414 [1e-7 erg / (s cm^2 Ang)]
Narrow H alpha = 4403.8706 [1e-7 erg / (s cm^2 Ang)]


The H $\alpha$ flux values from the model and the observed spectrum are very close!

#### H $\beta$

In [80]:
# H beta
print('Model: ')
mask_H_beta = (s.wave > 4750) & (s.wave < 4950)

# broad
flux_Hb_broad = np.sum(broad_1(s.wave)[mask_H_beta])
print('Broad H beta = ' + str(flux_Hb_broad) + ' [1e-7 erg / (s cm^2 Ang)]')

# narrow
flux_Hb_narrow = np.sum(narrow_1(s.wave)[mask_H_beta])
print('Narrow H beta = ' + str(flux_Hb_narrow) + ' [1e-7 erg / (s cm^2 Ang)]')

Model: 
Broad H beta = 4595.4443 [1e-7 erg / (s cm^2 Ang)]
Narrow H beta = 771.61237 [1e-7 erg / (s cm^2 Ang)]


In [82]:
# H beta
print('Observed spectrum: ')

mask_H_beta_or = (s_or.wave > 4750) & (s_or.wave < 4950)

# broad
flux_Hb_broad_or = np.sum(broad_1(s_or.wave)[mask_H_beta_or])
print('Broad H beta = ' + str(flux_Hb_broad_or) + ' [1e-7 erg / (s cm^2 Ang)]')

# narrow
flux_Hb_narrow_or = np.sum(narrow_1(s_or.wave)[mask_H_beta_or])
print('Narrow H beta = ' + str(flux_Hb_narrow_or) + ' [1e-7 erg / (s cm^2 Ang)]')

Observed spectrum: 
Broad H beta = 4595.8555 [1e-7 erg / (s cm^2 Ang)]
Narrow H beta = 773.1469 [1e-7 erg / (s cm^2 Ang)]


The H $\beta$ values are very close too.

#### Fe

In [83]:
print('Model: ')

flux_feII = np.sum(fe_2(s.wave))
print('Total FeII = ' + str(flux_feII) + ' [1e-7 erg / (s cm^2 Ang)]')

Model: 
Total FeII = 4297.4854 [1e-7 erg / (s cm^2 Ang)]


In [84]:
print('Observed spectrum:')

flux_feII_or = np.sum(fe_2(s_or.wave))
print('Total FeII = ' + str(flux_feII_or) + ' [1e-7 erg / (s cm^2 Ang)]')

Observed spectrum:
Total FeII = 4284.889 [1e-7 erg / (s cm^2 Ang)]


The iron lines differ but in less than 10 flux units (1e-7 erg / (s cm^2 Ang)).

#### [OIII]

In [79]:
print('Model:')

mask_oiii = (s.wave > 4250) & (s.wave < 4450)
flux_oiii = np.sum(narrow_1(s.wave)[mask_oiii])
print('Total OIII (at 4364 ang) = ' + str(flux_oiii) + ' [1e-7 erg / (s cm^2 Ang)]')

mask_oiii_5007 = (s.wave > 4975) & (s.wave < 5050)
flux_oiii_5007 = np.sum(narrow_1(s.wave)[mask_oiii_5007])
print('Total OIII (at 5007 ang) = ' + str(flux_oiii_5007) + ' [1e-7 erg / (s cm^2 Ang)]')

Model:
Total OIII (at 4364 ang) = 349.4876 [1e-7 erg / (s cm^2 Ang)]
Total OIII (at 5007 ang) = 778.99664 [1e-7 erg / (s cm^2 Ang)]


In [85]:
print('Observed spectrum: ')

mask_oiii_or = (s_or.wave > 4250) & (s_or.wave < 4450)
flux_oiii_or = np.sum(narrow_1(s_or.wave)[mask_oiii_or])
print('Total OIII (at 4364 ang) = ' + str(flux_oiii_or) + ' [1e-7 erg / (s cm^2 Ang)]')

mask_oiii_5007_or = (s_or.wave > 4975) & (s_or.wave < 5050)
flux_oiii_5007_or = np.sum(narrow_1(s_or.wave)[mask_oiii_5007_or])
print('Total OIII (at 5007 ang) = ' + str(flux_oiii_5007_or) + ' [1e-7 erg / (s cm^2 Ang)]')

Observed spectrum: 
Total OIII (at 4364 ang) = 349.4848 [1e-7 erg / (s cm^2 Ang)]
Total OIII (at 5007 ang) = 778.5438 [1e-7 erg / (s cm^2 Ang)]


In the [OIII] line, the spectrum and model have very similar values.

#### [SIII]

In [86]:
print('Model: ')

mask_siii = (s.wave > 6670) & (s.wave < 6750)
flux_siii = np.sum(narrow_1(s.wave)[mask_siii])
print('Total SIII = ' + str(flux_siii) + ' [1e-7 erg / (s cm^2 Ang)]')

Model: 
Total SIII = 182.47621 [1e-7 erg / (s cm^2 Ang)]


In [88]:
print('Observed spectrum: ')

mask_siii_or = (s_or.wave > 6670) & (s_or.wave < 6750)
flux_siii_or = np.sum(narrow_1(s_or.wave)[mask_siii_or])
print('Total SIII = ' + str(flux_siii_or) + ' [1e-7 erg / (s cm^2 Ang)]')

Observed spectrum: 
Total SIII = 182.63913 [1e-7 erg / (s cm^2 Ang)]


The [SIII] line is quite faint, but can be identified in the plot. The model's and observed values are almost the same.

However, the [NII] line is not present.

# Calculating Line Ratios

I did not calculate the [NII]6583 / H $\alpha$ ratio as the [NII] line is not present.

## [OIII]5007 / H $\beta$

In [89]:
print('Model: ')

oiii_Hbeta_broad = flux_oiii_5007 / flux_Hb_broad
print('[OIII] / Broad H beta = ' + str(oiii_Hbeta_broad))

oiii_Hbeta_narrow = flux_oiii_5007 / flux_Hb_narrow
print('[OIII] / Narrow H beta = ' + str(oiii_Hbeta_narrow))

Model: 
[OIII] / Broad H beta = 0.16951498
[OIII] / Narrow H beta = 1.0095699


In [90]:
print('Observed spectrum: ')

oiii_Hbeta_broad_or = flux_oiii_5007_or / flux_Hb_broad_or
print('[OIII] / Broad H beta = ' + str(oiii_Hbeta_broad_or))

oiii_Hbeta_narrow_or = flux_oiii_5007_or / flux_Hb_narrow_or
print('[OIII] / Narrow H beta = ' + str(oiii_Hbeta_narrow_or))

Observed spectrum: 
[OIII] / Broad H beta = 0.16940129
[OIII] / Narrow H beta = 1.0069804


## H $\alpha$ / H $\beta$

In [91]:
print('Model: ')

Ha_Hb_broad = flux_Ha_broad / flux_Hb_broad
print('Broad H alpha / H beta = ' + str(Ha_Hb_broad))

Ha_Hb_narrow = flux_Ha_narrow / flux_Hb_narrow
print('Narrow H alpha / H beta = ' + str(Ha_Hb_narrow))

Model: 
Broad H alpha / H beta = 1.7105275
Narrow H alpha / H beta = 5.710056


In [92]:
print('Observed spectra: ')

Ha_Hb_broad_or = flux_Ha_broad_or / flux_Hb_broad_or
print('Broad H alpha / H beta = ' + str(Ha_Hb_broad_or))

Ha_Hb_narrow_or = flux_Ha_narrow_or / flux_Hb_narrow_or
print('Narrow H alpha / H beta = ' + str(Ha_Hb_narrow_or))

Observed spectra: 
Broad H alpha / H beta = 1.7101091
Narrow H alpha / H beta = 5.6960335


The ratios for the corrected/modeled spectrum and the non-corrected/observed one are very close, and they differ only by ~0.01 or less.

**References**

1. [Rakic, N., 2022, MNRAS, 516, 1624](https://ui.adsabs.harvard.edu/abs/2022MNRAS.516.1624R/abstract)
2. [Ilic, D. et al. 2020, A&A, 638, 13](https://ui.adsabs.harvard.edu/abs/2020A%26A...638A..13I/abstract)
3. [Kovacevic, J. et al. 2010, ApJS, 189, 15](https://ui.adsabs.harvard.edu/abs/2010ApJS..189...15K/abstract)
4. [Shapovalova, A. I. et al. 2012, ApJS, 202, 10](https://ui.adsabs.harvard.edu/abs/2012ApJS..202...10S/abstract)
5. Ilic, D., Rakic, N., Popovic, L. C., 2023, submitted